In [4]:
import subprocess
import os

import pandas as pd

import requests
from bs4 import BeautifulSoup

import json



pd.set_option('display.max_rows', 500)

from importlib import reload
import sys
reload(sys)
sys.setdefaultencoding("utf-8")

AttributeError: module 'sys' has no attribute 'setdefaultencoding'

![CRISP_DM](../reports/figures/CRISP_DM.png)

# Data Understanding

* RKI, webscrape (webscraping) https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Fallzahlen.html
* John Hopkins (GITHUB) https://github.com/CSSEGISandData/COVID-19.git
* REST API services to retreive data https://npgeo-corona-npgeo-de.hub.arcgis.com/

## GITHUB csv data

git clone/pull https://github.com/CSSEGISandData/COVID-19.git

In [6]:

git_pull = subprocess.Popen( "/usr/bin/git pull" , 
                     cwd = os.path.dirname( '../data/raw/COVID-19/' ), 
                     shell = True, 
                     stdout = subprocess.PIPE, 
                     stderr = subprocess.PIPE )
(out, error) = git_pull.communicate()


print("Error : " + str(error)) 
print("out : " + str(out))

Error : 
out : Already up-to-date.



In [9]:
data_path='../data/raw/COVID-19/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
pd_raw=pd.read_csv(data_path)

In [10]:
pd_raw.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,8/13/20,8/14/20,8/15/20,8/16/20,8/17/20,8/18/20,8/19/20,8/20/20,8/21/20,8/22/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,37424,37431,37551,37596,37599,37599,37599,37856,37894,37953
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,6971,7117,7260,7380,7499,7654,7812,7967,8119,8275
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,37187,37664,38133,38583,39025,39444,39847,40258,40667,41068
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,981,989,989,989,1005,1005,1024,1024,1045,1045
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,1815,1852,1879,1906,1935,1966,2015,2044,2068,2134


## Webscrapping

In [11]:
page = requests.get("https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Fallzahlen.html")

In [12]:
soup = BeautifulSoup(page.content, 'html.parser')

In [13]:
html_table=soup.find('table') # find the table, attention this works if one table exists

In [14]:
all_rows=html_table.find_all('tr')

In [15]:
final_data_list=[]

In [16]:
for pos,rows in enumerate(all_rows):
   
    col_list=[each_col.get_text(strip=True) for each_col in rows.find_all('td')] #td for data element
    final_data_list.append(col_list)
    

In [17]:
pd_daily_status=pd.DataFrame(final_data_list).dropna().rename(columns={0:'state',
                                                       1:'cases',
                                                       2:'changes',
                                                       3:'cases_per_100k',
                                                       4:'fatal',
                                                       5:'comment'})

In [18]:
pd_daily_status.head()

,state,cases,changes,cases_per_100k,fatal,comment
2,Baden-Württem­berg,40.031,+202,1.369,"12,4",1.863
3,Bayern,54.858,+41,1.817,"13,9",2.633
4,Berlin,10.717,+62,450,"12,0",226
5,Branden­burg,3.787,+3,83,"3,3",169
6,Bremen,1.900,+7,59,"8,6",56


## REST API calls

In [19]:
## data request for Germany
data=requests.get('https://services7.arcgis.com/mOBPykOjAyBO2ZKk/arcgis/rest/services/Coronaf%C3%A4lle_in_den_Bundesl%C3%A4ndern/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json')

In [20]:
json_object=json.loads(data.content) 

In [21]:
type(json_object)

dict

In [22]:
json_object.keys()

[u'geometryProperties',
 u'uniqueIdField',
 u'fields',
 u'spatialReference',
 u'geometryType',
 u'objectIdFieldName',
 u'globalIdFieldName',
 u'features']

In [32]:
full_list=[]
for pos,each_dict in enumerate (json_object['features'][:]):
    full_list.append(each_dict['attributes'])
    

In [33]:
pd_full_list=pd.DataFrame(full_list)
pd_full_list.head()

,AGS_TXT,Aktualisierung,Death,Fallzahl,GlobalID,LAN_ew_AGS,LAN_ew_BEZ,LAN_ew_EWZ,LAN_ew_GEN,OBJECTID,OBJECTID_1,Shape__Area,Shape__Length,faelle_100000_EW
0,01,1598133600000,160,3902,fc5ba936-c95c-432c-8a33-9eb2f30b660f,01,Land,2896712,Schleswig-Holstein,15,1,4.573731e+10,2.881496e+06,134.704451
1,02,1598133600000,265,6019,0f3e860c-5181-4d3f-a421-1d51f50315ea,02,Freie und Hansestadt,1841179,Hamburg,6,2,2.089396e+09,4.188002e+05,326.910094
2,03,1598133600000,659,16055,3fd77024-c29b-4843-9be8-682ad48e60c9,03,Land,7982448,Niedersachsen,9,3,1.299836e+11,4.008988e+06,201.128777
3,04,1598133600000,56,1900,4132268b-54de-4327-ac1e-760e915112f1,04,Freie Hansestadt,682986,Bremen,5,4,1.119157e+09,3.357177e+05,278.190183
4,05,1598133600000,1798,56784,561d658f-3ee5-46e3-bc95-3528c6558ab9,05,Land,17932651,Nordrhein-Westfalen,10,5,8.782936e+10,2.648673e+06,316.651453


In [36]:
pd_full_list.to_csv('../data/raw/NPGEO/GER_state_data.csv',sep=';')

In [37]:
pd_full_list.shape[0]

16

# API access via REST service, e.g. USA data 

example of a REST conform interface (attention registration mandatory)

www.smartable.ai

In [41]:


# US for full list
headers = {
    'Cache-Control': 'no-cache',
    'Subscription-Key': '28ee4219700f48718be78b057beb7eb4',
}

response = requests.get('https://api.smartable.ai/coronavirus/stats/US', headers=headers)
print(response)

In [42]:

US_dict=json.loads(response.content) # imports string
with open('../data/raw/SMARTABLE/US_data.json', 'w') as outfile:
    json.dump(US_dict, outfile,indent=2)

In [43]:
print(json.dumps(US_dict,indent=2)) #string dump

# Individual States US

In [44]:
US_dict['stats']['breakdowns'][0]

{u'location': {u'countryOrRegion': u'United States',
  u'county': None,
  u'isoCode': None,
  u'lat': 13.444304,
  u'long': 144.793731,
  u'provinceOrState': u'Guam'},
 u'newDeaths': 0,
 u'newlyConfirmedCases': 0,
 u'newlyRecoveredCases': 0,
 u'totalConfirmedCases': 32,
 u'totalDeaths': 1,
 u'totalRecoveredCases': 0}

In [45]:
full_list_US_country=[]
for pos,each_dict in enumerate (US_dict['stats']['breakdowns'][:]):
    flatten_dict=each_dict['location']
    flatten_dict.update(dict(list(US_dict['stats']['breakdowns'][pos].items())[1: 7]) 
    )
    full_list_US_country.append(flatten_dict)

In [46]:
pd.DataFrame(full_list_US_country).to_csv('../data/raw/SMARTABLE/full_list_US_country.csv',sep=';',index=False)